<h1><center>Entity Recognition & Sentiment Analysis</center></h1>

# Entity Recognition of Tweets

In [ ]:
#Connecting to the MongoDB database 'EngTweetsDb' where the tweets are stored
import pymongo
MONGO_HOST = 'mongodb://localhost/EngTweetsDb'
conn = pymongo.MongoClient(MONGO_HOST)["EngTweetsDb"]["en_tweets_col"]
conn.count()

In [ ]:
#Using 'NLTK' package to perform entity recognition
import os
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

In [ ]:
#Setting the necessary path
java_path = "C:/Program Files/Java/jdk1.8.0_161/bin/java.exe"
os.environ['JAVAHOME'] = java_path

In [ ]:
#Invoking an instance of 'StanfordNERTagger'
st = StanfordNERTagger('C:/Users/Dell user/Desktop/Stanford/stanford-ner-2018-02-27/classifiers/english.conll.4class.distsim.crf.ser.gz',
                       'C:/Users/Dell user/Desktop/Stanford/stanford-ner-2018-02-27/stanford-ner.jar',
                       encoding='utf-8')

In [ ]:
#Creating a new MongoDB databse 'Entity' to store entities
MONGO_HOST1 = 'mongodb://localhost/Entity'
conn1 = pymongo.MongoClient(MONGO_HOST1)

In [ ]:
#Creating a function to extract the entities
def entity_recognizer(text):
    entity = st.tag(txt.split())
    record = {'entity': entity}
    db1.entity_record.save(record)
    return entity

In [ ]:
#Extracting only the tweet text from the database
all_text_id =(list(conn.find({},{"text":1})))
text_only = []
for i in range(conn.count()):
    text_str = all_text_id[i]["text"]
    text_only.append(text_str)
print(text_only)

In [ ]:
#Calling 'entity_recognizer' function on the tweet text
db1 = conn1.Entity
entity_list = []
entity_list_nw = []
for txt in text_only:
    entity = entity_recognizer(txt)
    entity_list.append(entity)
    entity_list_nw = entity_list_nw + entity
print(entity_list_nw)

In [ ]:
#Converting the tweet text into lowercase
l = [(item[0].lower(),item[1]) for item in entity_list_nw]
print(l)

In [ ]:
#Using 'collections' package to count the number of entities categorised as 'PERSON', 'ORGANISATION' and 'LOCATION'
from collections import Counter
print(Counter(x[0] for x in l if x[1]!='O' ))
print(Counter(x[1] for x in l if x[1]!='O' ))
counts = Counter(x[0] for x in l if x[1]!='O')
print(counts.most_common())

In [ ]:
#Extracting the top-five entities
top_entities=list(Counter(x[0] for x in counts.most_common() if x[1]!='O'))
top_five = top_entities[0:5]

In [ ]:
#Printing 'top_five' list of entities
top_five

# Sentiment Analysis

## (A) Tweets

In [ ]:
#Using 'textblob' package to do find out polarity of the tweet text which is then used to categorize tweets as positive, neutral or negative.
from textblob import TextBlob
def get_tweet_sentiment(twt):
        
        analysis = TextBlob(twt)
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [ ]:
#Creating a list of tuples containing the text and the sentiment: positive/neutral/negative
analysis = []
for twt in text_only:
    senti = (twt, get_tweet_sentiment(twt))
    analysis.append(senti)
    
print(analysis)

In [ ]:
#Printing the percentage of positive, neutral and negative sentiments in all the tweets
counts = Counter(x[1] for x in analysis)
print('No. of positive sentiments    = {}'.format(counts['positive']))
print('No. of neutral sentiments     = {}'.format(counts['neutral']))
print('No. of negative sentiments    = {}'.format(counts['negative']))
print('Total tweets analysed         = {}'.format(len(analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts['positive']/len(analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts['neutral']/len(analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts['negative']/len(analysis))))


## (B) News Articles

In [ ]:
#Connecting to the MongoDB database 'News' where the news articles corresponding to five most-occurring entities are stored
import pymongo
MONGO_HOST = 'mongodb://localhost/News'
conn2 = pymongo.MongoClient(MONGO_HOST)["News"]
db2 = conn1.articles

In [ ]:
#Passing the list of five most frequently occurring entities in tweets
top_five = ['hindu', 'justin', 'bieber', '#bbmas', 'trump']

In [ ]:
#Extracting the 'content' information from the stored news articles
content_id =(list(db2.find({})))

In [ ]:
#Creating a function to extract only the description/body of the news articles
def textprocessor(s):
    start="'body':"
    end="'categories':"
    body=(s.split(start))[1].split(end)[0]
    return body.strip()

In [ ]:
#Storing content of news article corresponding to each entity of 'top_five' list in five different lists, namely, body1_list, body2_list, body3_list, body4_list and body5_list
body1_list = []
for i in range(0,10):
    s = content_id[i][top_five[0]]
    body1 = textprocessor(s)
    print(top_five[0])
    print(body1)
    print("\n")
    body1_list.append(body1)
    
body2_list = []    
for i in range(11,20):
    s = content_id[i][top_five[1]]
    body2 = textprocessor(s)
    print(top_five[1])
    print(body2)
    print("\n")
    body2_list.append(body2)
body3_list=[]    
for i in range(21,30):
    s = content_id[i][top_five[2]]
    body3 = textprocessor(s)
    print(top_five[2])
    print(body3)
    print("\n")
    body3_list.append(body3)
body4_list = []    
for i in range(31,40):
    s = content_id[i][top_five[3]]
    body4 = textprocessor(s)
    print(top_five[3])
    print(body4)
    print("\n")
    body4_list.append(body4)
body5_list = []    
for i in range(41,50):
    s = content_id[i][top_five[4]]
    body5 = textprocessor(s)
    print(top_five[4])
    print(body5)
    print("\n")
    body5_list.append(body5)

In [ ]:
#Creating a function to extract the entities
db3 = conn2.content_entity_record
def entity_recognizer_news(text):
    entity = st.tag(txt.split(' '))
    print(entity)
    record = {'entity': entity}
    db3.content_entity_record.save(record)
    return entity

In [ ]:
#Using 'textblob' package to do find out polarity of the tweet text which is then used to categorize tweets as positive, neutral or negative.
from textblob import TextBlob
def get_news_sentiment(txt):
        
        analysis = TextBlob(txt)
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [ ]:
#Storing tuple of content and its sentiment in lists, namely, body1_analysis, body2_analysis, body3_analysis, body4_analysis and body5_analysis 
body1_analysis = []
for txt in body1_list:
    senti = (get_news_sentiment(txt), txt)
    body1_analysis.append(senti)
print(body1_analysis)

body2_analysis = []
for txt in body2_list:
    senti = (get_news_sentiment(txt), txt)
    body2_analysis.append(senti)
print(body2_analysis)

body3_analysis = []
for txt in body3_list:
    senti = (get_news_sentiment(txt), txt)
    body3_analysis.append(senti)
print(body3_analysis)

body4_analysis = []
for txt in body4_list:
    senti = (get_news_sentiment(txt), txt)
    body4_analysis.append(senti)
print(body4_analysis)

body5_analysis = []
for txt in body5_list:
    senti = (get_news_sentiment(txt), txt)
    body5_analysis.append(senti)
print(body5_analysis)

In [ ]:
#Printing the percentage of positive, neutral and negative sentiments for each entity list, namely, body1_analysis, body2_analysis, body3_analysis, body4_analysis and body5_analysis
from collections import Counter

counts6 = Counter(x[0] for x in body1_analysis)
print(top_five[0])
print('No. of positive sentiments    = {}'.format(counts6['positive']))
print('No. of neutral sentiments     = {}'.format(counts6['neutral']))
print('No. of negative sentiments    = {}'.format(counts6['negative']))
print('Total tweets analysed         = {}'.format(len(body1_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts6['positive']/len(body1_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts6['neutral']/len(body1_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts6['negative']/len(body1_analysis))))
print('\n')

counts7 = Counter(x[0] for x in body2_analysis)
print(top_five[1])
print('No. of positive sentiments    = {}'.format(counts7['positive']))
print('No. of neutral sentiments     = {}'.format(counts7['neutral']))
print('No. of negative sentiments    = {}'.format(counts7['negative']))
print('Total tweets analysed         = {}'.format(len(body2_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts7['positive']/len(body2_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts7['neutral']/len(body2_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts7['negative']/len(body2_analysis))))
print('\n')

counts8 = Counter(x[0] for x in body3_analysis)
print(top_five[2])
print('No. of positive sentiments    = {}'.format(counts8['positive']))
print('No. of neutral sentiments     = {}'.format(counts8['neutral']))
print('No. of negative sentiments    = {}'.format(counts8['negative']))
print('Total tweets analysed         = {}'.format(len(body3_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts8['positive']/len(body3_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts8['neutral']/len(body3_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts8['negative']/len(body3_analysis))))
print('\n')

counts9 = Counter(x[0] for x in body4_analysis)
print(top_five[3])
print('No. of positive sentiments    = {}'.format(counts9['positive']))
print('No. of neutral sentiments     = {}'.format(counts9['neutral']))
print('No. of negative sentiments    = {}'.format(counts9['negative']))
print('Total tweets analysed         = {}'.format(len(body4_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts9['positive']/len(body4_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts9['neutral']/len(body4_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts9['negative']/len(body4_analysis))))
print('\n')

counts10 = Counter(x[0] for x in body5_analysis)
print(top_five[4])
print('No. of positive sentiments    = {}'.format(counts10['positive']))
print('No. of neutral sentiments     = {}'.format(counts10['neutral']))
print('No. of negative sentiments    = {}'.format(counts10['negative']))
print('Total tweets analysed         = {}'.format(len(body5_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts10['positive']/len(body5_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts10['neutral']/len(body5_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts10['negative']/len(body5_analysis))))

## (C) Finding and analysing sentiment of tweets corresponding to each entity in 'top_five' list

In [ ]:
#Connecting to the MongoDB database 'EngTweetsDb' where the tweets are stored
client = pymongo.MongoClient(MONGO_HOST)
db1 = client.EngTweetsDb
db1.en_tweets_col.find_one()

In [ ]:
#Finding tweets corresponding to each entity in 'top_five' list and storing it in list
one_tweets = db1.en_tweets_col.find({"text" : { '$regex' : 'hindu', '$options':'i'}})
one_only = []
for i in range(one_tweets.count()):
    one_str = one_tweets[i]['text']
    one_only.append(one_str)
print(one_only)

two_tweets = db1.en_tweets_col.find({"text" : { '$regex' : 'justin', '$options':'i'}})
two_only = []
for i in range(two_tweets.count()):
    two_str = two_tweets[i]['text']
    two_only.append(two_str)
print(two_only)

three_tweets = db1.en_tweets_col.find({"text" : { '$regex' : 'bieber', '$options':'i'}})
three_only = []
for i in range(three_tweets.count()):
    three_str = three_tweets[i]['text']
    three_only.append(three_str)
print(three_only)

four_tweets = db1.en_tweets_col.find({"text" : { '$regex' : '#bbmas', '$options':'i'}})
four_only = []
for i in range(four_tweets.count()):
    four_str = four_tweets[i]['text']
    four_only.append(four_str)
print(four_only)

five_tweets = db1.en_tweets_col.find({"text" : { '$regex' : top_five[4], '$options':'i'}})
five_only = []
for i in range(five_tweets.count()):
    five_str = five_tweets[i]['text']
    five_only.append(five_str)
print(five_only)

In [ ]:
#Storing tuple of tweet text and its sentiment in lists, namely, one_analysis, two_analysis, three_analysis, four_analysis and five_analysis 
one_analysis = []
for txt in one_only:
    senti = (get_tweet_sentiment(txt), txt)
    one_analysis.append(senti)
print(one_analysis)

two_analysis = []
for txt in two_only:
    senti = (get_tweet_sentiment(txt), txt)
    two_analysis.append(senti)    
print(two_analysis)

three_analysis = []
for txt in three_only:
    senti = (get_tweet_sentiment(txt), txt)
    three_analysis.append(senti)    
print(three_analysis)

four_analysis = []
for txt in four_only:
    senti = (get_tweet_sentiment(txt), txt)
    four_analysis.append(senti)    
print(four_analysis)

five_analysis = []
for txt in five_only:
    senti = (get_tweet_sentiment(txt), txt)
    five_analysis.append(senti)    
print(five_analysis)

In [ ]:
#Printing the percentage of positive, neutral and negative sentiments for each entity list, namely, one_analysis, two_analysis, three_analysis, four_analysis and five_analysis

from collections import Counter

counts1 = Counter(x[0] for x in one_analysis)
print(top_five[0])
print('No. of positive sentiments    = {}'.format(counts1['positive']))
print('No. of neutral sentiments     = {}'.format(counts1['neutral']))
print('No. of negative sentiments    = {}'.format(counts1['negative']))
print('Total tweets analysed         = {}'.format(len(one_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts1['positive']/len(one_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts1['neutral']/len(one_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts1['negative']/len(one_analysis))))
print('\n')

counts2 = Counter(x[0] for x in two_analysis)
print(top_five[1])
print('No. of positive sentiments    = {}'.format(counts2['positive']))
print('No. of neutral sentiments     = {}'.format(counts2['neutral']))
print('No. of negative sentiments    = {}'.format(counts2['negative']))
print('Total tweets analysed         = {}'.format(len(two_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts2['positive']/len(two_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts2['neutral']/len(two_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts2['negative']/len(two_analysis))))
print('\n')

counts3 = Counter(x[0] for x in three_analysis)
print(top_five[2])
print('No. of positive sentiments    = {}'.format(counts3['positive']))
print('No. of neutral sentiments     = {}'.format(counts3['neutral']))
print('No. of negative sentiments    = {}'.format(counts3['negative']))
print('Total tweets analysed         = {}'.format(len(three_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts3['positive']/len(three_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts3['neutral']/len(three_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts3['negative']/len(three_analysis))))
print('\n')

counts4 = Counter(x[0] for x in four_analysis)
print(top_five[3])
print('No. of positive sentiments    = {}'.format(counts4['positive']))
print('No. of neutral sentiments     = {}'.format(counts4['neutral']))
print('No. of negative sentiments    = {}'.format(counts4['negative']))
print('Total tweets analysed         = {}'.format(len(four_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts4['positive']/len(four_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts4['neutral']/len(four_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts4['negative']/len(four_analysis))))
print('\n')

counts5 = Counter(x[0] for x in five_analysis)
print(top_five[4])
print('No. of positive sentiments    = {}'.format(counts5['positive']))
print('No. of neutral sentiments     = {}'.format(counts5['neutral']))
print('No. of negative sentiments    = {}'.format(counts5['negative']))
print('Total tweets analysed         = {}'.format(len(five_analysis)))
print('Percentage of positive tweets = {0:10.3f} %'.format((100*counts5['positive']/len(five_analysis))))
print('Percentage of neutral tweets  = {0:10.3f} %'.format((100*counts5['neutral']/len(five_analysis))))
print('Percentage of negative tweets = {0:10.3f} %'.format((100*counts5['negative']/len(five_analysis))))

# Comparison between sentiments of tweets and news articles for named-entities

## (A) Entity 1: Hindu

In [ ]:
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts1['positive']/len(one_analysis)), (100*counts1['neutral']/len(one_analysis)),(100*counts1['negative']/len(one_analysis))],
    name='Tweets'
)
trace2 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts6['positive']/len(body1_analysis)),(100*counts6['neutral']/len(body1_analysis)),(100*counts6['negative']/len(body1_analysis))],
    name='News Articles'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='Entity 1: Hindu',
    yaxis=dict(
        title='Percentage (%)')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Entity 2: Justin

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts2['positive']/len(two_analysis)), (100*counts2['neutral']/len(two_analysis)),(100*counts2['negative']/len(two_analysis))],
    name='Tweets'
)
trace2 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts7['positive']/len(body2_analysis)),(100*counts7['neutral']/len(body2_analysis)),(100*counts7['negative']/len(body2_analysis))],
    name='News Articles'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='Entity 2: Justin',
    yaxis=dict(
        title='Percentage (%)')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Entity 3: Bieber

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts3['positive']/len(three_analysis)), (100*counts3['neutral']/len(three_analysis)),(100*counts3['negative']/len(three_analysis))],
    name='Tweets'
)
trace2 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts8['positive']/len(body3_analysis)),(100*counts8['neutral']/len(body3_analysis)),(100*counts8['negative']/len(body3_analysis))],
    name='News Articles'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='Entity 3: Bieber',
    yaxis=dict(
        title='Percentage (%)')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Entity 4: #bbmas

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts4['positive']/len(four_analysis)), (100*counts4['neutral']/len(four_analysis)),(100*counts4['negative']/len(four_analysis))],
    name='Tweets'
)
trace2 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts9['positive']/len(body4_analysis)),(100*counts9['neutral']/len(body4_analysis)),(100*counts9['negative']/len(body4_analysis))],
    name='News Articles'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='Entity 4: #bbmas',
    yaxis=dict(
        title='Percentage (%)')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Entity 5: Trump

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts5['positive']/len(five_analysis)), (100*counts5['neutral']/len(five_analysis)),(100*counts5['negative']/len(five_analysis))],
    name='Tweets'
)
trace2 = go.Bar(
    x=['positive', 'neutral', 'negative'],
    y=[(100*counts10['positive']/len(body5_analysis)),(100*counts10['neutral']/len(body5_analysis)),(100*counts10['negative']/len(body5_analysis))],
    name='News Articles'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='Entity 5: Trump',
    yaxis=dict(
        title='Percentage (%)')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)